## Verisetinin Yüklenmesi

<span style="font-size:1.1em;">Colab'a Google drive'ı entegre ediyoruz. Kullanılacak olan veriseti Google Drive'da bulunmaktadır</span>

In [203]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
PREPROCESSED_DATASET_WITH_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_with_stemming.csv"
PREPROCESSED_DATASET_WITHOUT_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_no_stemming.csv"
PREPROCESSED_DATASET_ZEMBEREK = "gdrive/My Drive/mbti/preprocessed_dataset_zemberek.csv"
TRIMMED_DATASET = "gdrive/My Drive/mbti/trimmed_dataset.csv"
RAW_DATASET = "gdrive/My Drive/mbti/all_users_v2.csv"

<span style="font-size:1.1em;">Hangi veriseti kullanılarak işlem yapılacaksa yukardaki pathlerden biri seçilir ve parametre olarak verilir.</span>

In [0]:
import pandas as pd 
df = pd.read_csv(PREPROCESSED_DATASET_ZEMBEREK, sep = ';', header = 0)

In [206]:
df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,19991991,ekşi itiraf dön dolaş gel kendi çat problem ge...,ENTJ,analysts,E,N,T,J
1,19991991,selda bağcan ses dinle dinleyebil dinleyebilme...,ENTJ,analysts,E,N,T,J
2,19991991,eski sevgili mutlu ol olma iste isteyen insan ...,ENTJ,analysts,E,N,T,J
3,19991991,veda et ederken not bırak bırakmak fark farklı...,ENTJ,analysts,E,N,T,J
4,19991991,ingiliz aksa ara bayıl bayıldık konuş konuşan ...,ENTJ,analysts,E,N,T,J
...,...,...,...,...,...,...,...,...
524797,zaimoglu,zlatan ıbrahimovic türkiye katil,ESFJ,sentinels,E,S,F,J
524798,zaimoglu,tarih tarihteki büyük yalan yalancı şike opera...,ESFJ,sentinels,E,S,F,J
524799,zaimoglu,akp chp koalisyon hayal koalisyon,ESFJ,sentinels,E,S,F,J
524800,zaimoglu,trabzon trabzonlu insan hamsi de diyen fenerba...,ESFJ,sentinels,E,S,F,J


In [0]:
import numpy as np

df['entry'] = df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

Entrylerin sınıflarına eşit dağıtılması sağlanır.

In [208]:
min_entry = df.groupby('typeClass', as_index = False).count().min().entry

min_entry

54912

In [209]:
analysts_df = df[df['typeClass'] == 'analysts']
analysts_df = analysts_df.iloc[0 : min_entry]

analysts_df.shape[0]

54912

In [210]:
explorers_df = df[df['typeClass'] == 'explorers']
explorers_df = explorers_df.iloc[0 : min_entry]

explorers_df.shape[0]

54912

In [211]:
sentinels_df = df[df['typeClass'] == 'sentinels']
sentinels_df = sentinels_df.iloc[0 : min_entry]

sentinels_df.shape[0]

54912

In [212]:
diplomats_df = df[df['typeClass'] == 'diplomats']
diplomats_df = diplomats_df.iloc[0 : min_entry]

diplomats_df.shape[0]

54912

Yeni bir dataframe oluşturulur.

In [213]:
equal_entries_df = pd.concat([diplomats_df, sentinels_df, explorers_df, analysts_df]).reset_index(drop=True)
equal_entries_df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,1903 24,göt de den evrim gün soruştur soruşturma ıeee ...,INFJ,diplomats,I,N,F,J
1,1903 24,ekşi sözlük UNK club istanbul hazır,INFJ,diplomats,I,N,F,J
2,1903 24,UNK çöm sene süre yap info UNK çık sonuç aynı çık,INFJ,diplomats,I,N,F,J
3,1903 24,müzik dinle dinlemek haram uyuşturucu aşık der...,INFJ,diplomats,I,N,F,J
4,1903 24,zengin insan kur kurabil kurabilecek cümle al ...,INFJ,diplomats,I,N,F,J
...,...,...,...,...,...,...,...,...
219643,gulcehatun,america beauty izle izleyeli yıl rüya poşet rü...,INTJ,analysts,I,N,T,J
219644,gulcehatun,fedakar bencil birliktelik ortam bencil ol ola...,INTJ,analysts,I,N,T,J
219645,gulcehatun,kadın sürücü yol ver vermeme huy kadın yol ver...,INTJ,analysts,I,N,T,J
219646,gulcehatun,dövme baba öğret öğretme ömür yet yettik bisik...,INTJ,analysts,I,N,T,J


Oluşturulan dataframe shuffle edilir.

In [214]:
equal_entries_df = equal_entries_df.sample(frac=1).reset_index(drop=True)
equal_entries_df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,busefer,alaaddin varol basil savun savunduk anla anlam...,INFP,diplomats,I,N,F,P
1,hile yok limon var,öte t gönül gönüllü çağrı hastalık sebep müşah...,ESTJ,sentinels,E,S,T,J
2,melancohly,uzak mesafe ilişki ses ses değil bilgisayar ko...,ESTJ,sentinels,E,S,T,J
3,clostridium botulinum,eylül gsm operatör hat çök çökme deprem sonra ...,ESFP,explorers,E,S,F,P
4,nadyakomanaci,alevi alevilik alevi baskı tecrit kına katlet ...,ISTJ,sentinels,I,S,T,J
...,...,...,...,...,...,...,...,...
219643,nick kullanmak istemiyorum,formül vazgeç vazgeçil ikinci rakı yan yanında...,ESFP,explorers,E,S,F,P
219644,montaya,damla bilic in sahip çıkın çağrı kadın sözlük ...,ESFP,explorers,E,S,F,P
219645,altairibnlaahad,sözlük pop yazar sahtekar ece başlık haberdar ...,ENFJ,diplomats,E,N,F,J
219646,carbonifera,kadın vajina köpek balık beyin benzer benzerli...,INTJ,analysts,I,N,T,J


## Feature Extraction

<span style="font-size:1.1em;">TF-IDF özellik vektörünün çıkartılmasında kullanılacak değişken aşağıda belirlenmiş olan parametrelerle oluşturulur.</span>

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [0]:
count_vectorizer = CountVectorizer()

## Modelin Oluşturulması

<span style="font-size:1.1em;">Veriseti train ve test olmak üzere ikiye ayrılır. Test %20 ve train %80'ini oluşturacak şekilde tüm veriseti bölünür. random_state parametresi ile tekrardan bölündüğünde bir öncekiyle aynı train ve test veri setlerinin oluşturulması sağlanır.</span>

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['typeClass'], random_state = 42, test_size = 0.20)

Train ve test datasetlerinden tf-idf vektörleri çıkartılır


In [0]:
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_count)
X_train_tf = tf_transformer.transform(X_train_count)
X_test_tf = tf_transformer.transform(X_test_count)


Multinominal Naive Bayes modeli oluşturulur. Oluşturulan bu model verisetinde "type" olarak belirtilen "analysts", "diplomats", "sentimenls", "explorers" sınıflarından hangilerine ait olduğunu tahmin etmek için kullanılır

In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
clf = MultinomialNB().fit(X_train_tf, y_train)
test_typeClass = y_test.values

predictions = clf.predict(X_test_tf)

In [222]:
predictions

array(['sentinels', 'explorers', 'sentinels', ..., 'analysts',
       'sentinels', 'explorers'], dtype='<U9')

<span style="font-size:1.1em">Yapılacak tahminlerle ilgili istatistiksel verileri tutmak için</span> ```predictions_result```<span style="font-size:1.1em"> adında bir değişken oluşturulur.</span>

<span style="font-size:1.1em">Bu değişkenin yapısı aşağıdaki gibidir.</span>

```json
{
    "predicted": {
        "analysts":  { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "diplomats": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "explorers": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "sentinels": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
    }
}
```

* <span style="font-size:1.1em;">Yapılan tahminlerle ilgili verilere ulaşabilmek için</span>

    ```predictions_results['predicted']```


* <span style="font-size:1.1em;">Yapılan tahminin analyst ise:</span>

    ```predictions_results['predicted']['analysts']``` 


* <span style="font-size:1.1em;">Yapılan analyst tahmininin gerçek değerlerine erişmek için:</span>     

    ```predictions_results['predicted']['analysts']['actual']```  


* <span style="font-size:1.1em;">Test verisi, model tarafından analysts olarak tahmin edilmiştir ve bu verinin gerçek değeri de analysts'tir.</span>

    ```predictions_results['predicted']['analysts']['actual']['analysts']``` 

In [0]:
prediction_results = {'predicted': {}}  ## prediction_result['analysts'] means prediction is 'analysts'

prediction_results['predicted']['analysts']  = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['diplomats'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['explorers'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['sentinels'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}

## prediction_result['analysts']['diplomats'] means prediction is analysts but actual value is diplomats

```prediction_results```<span style="font-size:1.1em"> içerisinde tutulan sayaçların değerleri arttırılır.</span>

In [0]:
for i in range(len(predictions)):
  predicted_value = predictions[i]
  actual_value = test_typeClass[i]
  prediction_results['predicted'][predicted_value]['actual'][actual_value] += 1

<span style="font-size:1.1em">JSON formatına çevrilir </span>```dict``` <span style="font-size:1.1em">tipi. Bu sayede daha okunaklı bir şekilde print edilmiş olur. </span>

In [225]:
import json

print(json.dumps(prediction_results, indent = 2))

{
  "predicted": {
    "analysts": {
      "actual": {
        "analysts": 4155,
        "diplomats": 2664,
        "explorers": 2731,
        "sentinels": 2585
      }
    },
    "diplomats": {
      "actual": {
        "analysts": 1486,
        "diplomats": 2712,
        "explorers": 1463,
        "sentinels": 1329
      }
    },
    "explorers": {
      "actual": {
        "analysts": 2106,
        "diplomats": 2160,
        "explorers": 3498,
        "sentinels": 2051
      }
    },
    "sentinels": {
      "actual": {
        "analysts": 3248,
        "diplomats": 3349,
        "explorers": 3333,
        "sentinels": 5060
      }
    }
  }
}


<span style="font-size:1.1em;">İlgili field extract edilir</span> ```dict``` <span style="font-size:1.1em;">yapısından.</span>

In [0]:
results = prediction_results['predicted']

<span style="font-size:1.1em;">Başarı oranı hesaplanır</span>

In [227]:
accuracy = (results['analysts']['actual']['analysts'] + results['diplomats']['actual']['diplomats'] + results['explorers']['actual']['explorers'] + results['sentinels']['actual']['sentinels']) / len(predictions)
accuracy

0.3511267926246301

<span style="font-size:1.1em;">**E/I** boyutu tahmin edilir</span>

Entrylerin sınıflarına eşit dağıtılması sağlanır.

In [228]:
min_entry = df.groupby('I/E', as_index = False).count().min().entry

min_entry

258843

In [229]:
I_df = df[df['I/E'] == 'I']
I_df = I_df.iloc[0 : min_entry]

I_df.shape[0]

258843

In [230]:
E_df = df[df['I/E'] == 'E']
E_df = E_df.iloc[0 : min_entry]

E_df.shape[0]

258843

Yeni bir dataframe oluşturulur.

In [231]:
equal_entries_df = pd.concat([I_df, E_df]).reset_index(drop=True)
equal_entries_df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,Punisher 69,sözlük yazar kız ver verecek isim arap isim ol...,INTJ,analysts,I,N,T,J
1,Punisher 69,suriye suriyeli gönder vatandaş vatandaşlık ve...,INTJ,analysts,I,N,T,J
2,Punisher 69,klaus animasyon noel tema temalı film sev seve...,INTJ,analysts,I,N,T,J
3,Punisher 69,diamond tema zaman keşfet keşfettik youtube di...,INTJ,analysts,I,N,T,J
4,Punisher 69,fatih tezcan atatürk tweet atatürk sik yet ada...,INTJ,analysts,I,N,T,J
...,...,...,...,...,...,...,...,...
517681,serendiptytr,ramazan alan yemek iç içmek yasakla yasaklan i...,ESTJ,sentinels,E,S,T,J
517682,serendiptytr,fetö kelime anla anlat torba göz meymenet meym...,ESTJ,sentinels,E,S,T,J
517683,serendiptytr,gamze gamzeli erkek yıl yanak yağ tombul gül g...,ESTJ,sentinels,E,S,T,J
517684,serendiptytr,ince sahne at atıl atılan vatandaş özür dile d...,ESTJ,sentinels,E,S,T,J


Oluşturulan dataframe shuffle edilir.

In [232]:
equal_entries_df = equal_entries_df.sample(frac=1).reset_index(drop=True)
equal_entries_df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,tuzbuzz,dize dizelik şiir beş duy göz dol doldur doldu...,ENFP,diplomats,E,N,F,P
1,tuzbuzz,kitap kahve kedi yağmur müzik kusur sene vazge...,ENFP,diplomats,E,N,F,P
2,clostridium botulinum,beden don dondur dondurul dondurulan yaş yaşın...,ESFP,explorers,E,S,F,P
3,exodussian,avant window navigatör abant hotel kreatör var...,INTP,analysts,I,N,T,P
4,pier,eski thy çalış çalışan şok açıkla açıklama gün...,ESFJ,sentinels,E,S,F,J
...,...,...,...,...,...,...,...,...
517681,kanuni sozluk celebi,boy sevgili sar sarıl sarılarak uyu uyuyabil u...,ISFP,explorers,I,S,F,P
517682,bebek terlik,eylül kadıköy diren direniş sosyal medya köpek...,INTP,analysts,I,N,T,P
517683,radioheadbanger,radiohead çarşamba akşam parti var http UNK fa...,INFP,diplomats,I,N,F,P
517684,sagayaklisolbek,turgut özal odtü bilgisayar aç açıl açılış ded...,INFP,diplomats,I,N,F,P


In [0]:
X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['I/E'], random_state = 42)  ## Geri kalanlar S, T, J


X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_count)
X_train_tf = tf_transformer.transform(X_train_count)
X_test_tf = tf_transformer.transform(X_test_count)

clf = MultinomialNB().fit(X_train_tf, y_train)
test_typeClass = y_test.values

predictions = clf.predict(X_test_tf)

In [234]:
predictions

array(['I', 'I', 'E', ..., 'I', 'E', 'E'], dtype='<U1')

In [235]:
predicted = {}
predicted['I'] = {'actual': {'I': 0, 'E': 0}}
predicted['E'] = {'actual': {'I': 0, 'E': 0}}
predicted

{'E': {'actual': {'E': 0, 'I': 0}}, 'I': {'actual': {'E': 0, 'I': 0}}}

In [0]:
for i in range(len(predictions)):
  predicted[predictions[i]]['actual'][test_typeClass[i]] += 1


In [237]:
predicted

{'E': {'actual': {'E': 36972, 'I': 26543}},
 'I': {'actual': {'E': 27818, 'I': 38089}}}

In [0]:
accuracy = (predicted['E']['actual']['E'] + predicted['I']['actual']['I']) / len(predictions)

In [239]:
accuracy

0.5799709477523142

<span style="font-size:1.1em">**S/N** boyutu tahmin edilir.</span>

Entrylerin sınıflarına eşit dağıtılması sağlanır.

In [240]:
min_entry = df.groupby('S/N', as_index = False).count().min().entry

min_entry

147699

In [241]:
S_df = df[df['S/N'] == 'S']
S_df = S_df.iloc[0 : min_entry]

S_df.shape[0]

147699

In [242]:
N_df = df[df['S/N'] == 'N']
N_df = N_df.iloc[0 : min_entry]

N_df.shape[0]

147699

Yeni bir dataframe oluşturulur.

In [243]:
equal_entries_df = pd.concat([N_df, S_df]).reset_index(drop=True)
equal_entries_df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,19991991,ekşi itiraf dön dolaş gel kendi çat problem ge...,ENTJ,analysts,E,N,T,J
1,19991991,selda bağcan ses dinle dinleyebil dinleyebilme...,ENTJ,analysts,E,N,T,J
2,19991991,eski sevgili mutlu ol olma iste isteyen insan ...,ENTJ,analysts,E,N,T,J
3,19991991,veda et ederken not bırak bırakmak fark farklı...,ENTJ,analysts,E,N,T,J
4,19991991,ingiliz aksa ara bayıl bayıldık konuş konuşan ...,ENTJ,analysts,E,N,T,J
...,...,...,...,...,...,...,...,...
295393,zaimoglu,zlatan ıbrahimovic türkiye katil,ESFJ,sentinels,E,S,F,J
295394,zaimoglu,tarih tarihteki büyük yalan yalancı şike opera...,ESFJ,sentinels,E,S,F,J
295395,zaimoglu,akp chp koalisyon hayal koalisyon,ESFJ,sentinels,E,S,F,J
295396,zaimoglu,trabzon trabzonlu insan hamsi de diyen fenerba...,ESFJ,sentinels,E,S,F,J


Oluşturulan dataframe shuffle edilir.

In [244]:
equal_entries_df = equal_entries_df.sample(frac=1).reset_index(drop=True)
equal_entries_df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,lmrvnx,ar öğrenci avuç sopa vurul vurulma ora orda ya...,ISTJ,sentinels,I,S,T,J
1,kendisinin bir ucu,evlen evlenmeyen insan ten planla planladık ev...,ISTJ,sentinels,I,S,T,J
2,hayrullah amazingogullari,filip UNK benfica yıl euro bant oku okut okuta...,ESTJ,sentinels,E,S,T,J
3,sirinana,UNK öğren öğrenme olay çöz çözdük denge sağla ...,INTP,analysts,I,N,T,P
4,esseksurprizi,meral akşener in kedi sahiplen sahiplenme keşk...,ENTP,analysts,E,N,T,P
...,...,...,...,...,...,...,...,...
295393,buzkran,peder nick imparator imparatorluk bura burda s...,ENTP,analysts,E,N,T,P
295394,kendisinin bir ucu,terk et edil erkek gerçekten sev hal hoşlan ha...,ISTJ,sentinels,I,S,T,J
295395,metket,izmir bira midye zirve zirve katılım üst seviy...,INTP,analysts,I,N,T,P
295396,exodussian,uyur kreş bırak bırakıl bırakılan çocuk sene t...,INTP,analysts,I,N,T,P


In [0]:
X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['S/N'], random_state = 42)  ## Geri kalan boyutlar: T, J

X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_count)
X_train_tf = tf_transformer.transform(X_train_count)
X_test_tf = tf_transformer.transform(X_test_count)


clf = MultinomialNB().fit(X_train_tf, y_train)
test_typeClass = y_test.values

predictions = clf.predict(X_test_tf)

In [246]:
predicted['N'] = {'actual': {'N': 0, 'S': 0}}
predicted['S'] = {'actual': {'N': 0, 'S': 0}}

predicted

{'E': {'actual': {'E': 36972, 'I': 26543}},
 'I': {'actual': {'E': 27818, 'I': 38089}},
 'N': {'actual': {'N': 0, 'S': 0}},
 'S': {'actual': {'N': 0, 'S': 0}}}

In [0]:
for i in range(len(predictions)):
  predicted[predictions[i]]['actual'][test_typeClass[i]] += 1

In [248]:
predicted

{'E': {'actual': {'E': 36972, 'I': 26543}},
 'I': {'actual': {'E': 27818, 'I': 38089}},
 'N': {'actual': {'N': 20380, 'S': 13994}},
 'S': {'actual': {'N': 16470, 'S': 23006}}}

In [0]:
accuracy = (predicted['N']['actual']['N'] + predicted['S']['actual']['S']) / len(predictions)

In [250]:
accuracy

0.5874881516587678

<span style="font-size:1.1em">**T/F** boyutu tahmin edilir.</span>

Entrylerin sınıflarına eşit dağıtılması sağlanır.

In [251]:
min_entry = df.groupby('T/F', as_index = False).count().min().entry

min_entry

249807

In [252]:
T_df = df[df['T/F'] == 'T']
T_df = T_df.iloc[0 : min_entry]

T_df.shape[0]

249807

In [253]:
F_df = df[df['T/F'] == 'F']
F_df = F_df.iloc[0 : min_entry]

F_df.shape[0]

249807

Yeni bir dataframe oluşturulur.

In [254]:
equal_entries_df = pd.concat([F_df, T_df]).reset_index(drop=True)
equal_entries_df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,1903 24,göt de den evrim gün soruştur soruşturma ıeee ...,INFJ,diplomats,I,N,F,J
1,1903 24,ekşi sözlük UNK club istanbul hazır,INFJ,diplomats,I,N,F,J
2,1903 24,UNK çöm sene süre yap info UNK çık sonuç aynı çık,INFJ,diplomats,I,N,F,J
3,1903 24,müzik dinle dinlemek haram uyuşturucu aşık der...,INFJ,diplomats,I,N,F,J
4,1903 24,zengin insan kur kurabil kurabilecek cümle al ...,INFJ,diplomats,I,N,F,J
...,...,...,...,...,...,...,...,...
499609,sdthn,UNK go şey sat satıl satılamama sat satamayan ...,ISTJ,sentinels,I,S,T,J
499610,sdthn,bmw me oku okumak de mahmut de,ISTJ,sentinels,I,S,T,J
499611,sdthn,park araç silecek kır kıran adam sorun birey i...,ISTJ,sentinels,I,S,T,J
499612,sdthn,şey git gitme bıktır bıktıran dürüm iş düzel d...,ISTJ,sentinels,I,S,T,J


Oluşturulan dataframe shuffle edilir.

In [255]:
equal_entries_df = equal_entries_df.sample(frac=1).reset_index(drop=True)
equal_entries_df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,beyincik taciri,sözlük nick meslek tahmin ejder panda kungfu usta,ISFJ,sentinels,I,S,F,J
1,lejeuxdelamort,formula yüce tanrı kural değişik değişiklik de...,ENFJ,diplomats,E,N,F,J
2,lmrvnx,ıphone kullan insan ihtimal fakir,ISTJ,sentinels,I,S,T,J
3,cezm,şaka maka fenerbahçe los galacticos yıldız oyn...,INTJ,analysts,I,N,T,J
4,pixtur,UNK real market sat satıl almanya hasret çek ç...,ENTJ,analysts,E,N,T,J
...,...,...,...,...,...,...,...,...
499609,asmatavan,kadın koca aile kabullen kabullenmeme değişken...,ESFJ,sentinels,E,S,F,J
499610,yesilerikciktimi,suriye suriyeli git kal referandum nasrettin h...,ENFJ,diplomats,E,N,F,J
499611,macenas,eylül türkiye britanya basketbol maç melih mah...,INTJ,analysts,I,N,T,J
499612,blackout5,without fight brad paisley lovato UNK UNK yout...,ENFJ,diplomats,E,N,F,J


In [0]:
X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['T/F'], random_state = 42)  ##  J

X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_count)
X_train_tf = tf_transformer.transform(X_train_count)
X_test_tf = tf_transformer.transform(X_test_count)

clf = MultinomialNB().fit(X_train_tf, y_train)
test_typeClass = y_test.values

predictions = clf.predict(X_test_tf)

In [257]:
predicted['T'] = {'actual': {'T': 0, 'F': 0}}
predicted['F'] = {'actual': {'T': 0, 'F': 0}}

predicted

{'E': {'actual': {'E': 36972, 'I': 26543}},
 'F': {'actual': {'F': 0, 'T': 0}},
 'I': {'actual': {'E': 27818, 'I': 38089}},
 'N': {'actual': {'N': 20380, 'S': 13994}},
 'S': {'actual': {'N': 16470, 'S': 23006}},
 'T': {'actual': {'F': 0, 'T': 0}}}

In [0]:
for i in range(len(predictions)):
  predicted[predictions[i]]['actual'][test_typeClass[i]] += 1

In [259]:
predicted

{'E': {'actual': {'E': 36972, 'I': 26543}},
 'F': {'actual': {'F': 32957, 'T': 21370}},
 'I': {'actual': {'E': 27818, 'I': 38089}},
 'N': {'actual': {'N': 20380, 'S': 13994}},
 'S': {'actual': {'N': 16470, 'S': 23006}},
 'T': {'actual': {'F': 29540, 'T': 41037}}}

In [0]:
accuracy = (predicted['F']['actual']['F'] + predicted['T']['actual']['T']) / len(predictions)

In [261]:
accuracy

0.5924069685518478

<span style="font-size:1.1em">**J/P** boyutu tahmin edilir.</span>

Entrylerin sınıflarına eşit dağıtılması sağlanır.

In [262]:
min_entry = df.groupby('J/P', as_index = False).count().min().entry

min_entry

255859

In [263]:
J_df = df[df['J/P'] == 'J']
J_df = J_df.iloc[0 : min_entry]

J_df.shape[0]

255859

In [264]:
P_df = df[df['J/P'] == 'P']
P_df = P_df.iloc[0 : min_entry]

P_df.shape[0]

255859

Yeni bir dataframe oluşturulur.

In [265]:
equal_entries_df = pd.concat([P_df, J_df]).reset_index(drop=True)
equal_entries_df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,4yanlisyuzundengoturulendogru,video çek çekip giy giydik marka etiketle etik...,INTP,analysts,I,N,T,P
1,4yanlisyuzundengoturulendogru,iş yap para patron kazan hissi patron iste ist...,INTP,analysts,I,N,T,P
2,4yanlisyuzundengoturulendogru,öğren öğrenil öğrenildik ufuk kat çıkar çıkara...,INTP,analysts,I,N,T,P
3,4yanlisyuzundengoturulendogru,sevgili kg salla salla vur duvar,INTP,analysts,I,N,T,P
4,4yanlisyuzundengoturulendogru,kadın göğüs altın maya kok kokma temmuz ağusto...,INTP,analysts,I,N,T,P
...,...,...,...,...,...,...,...,...
511713,zaimoglu,zlatan ıbrahimovic türkiye katil,ESFJ,sentinels,E,S,F,J
511714,zaimoglu,tarih tarihteki büyük yalan yalancı şike opera...,ESFJ,sentinels,E,S,F,J
511715,zaimoglu,akp chp koalisyon hayal koalisyon,ESFJ,sentinels,E,S,F,J
511716,zaimoglu,trabzon trabzonlu insan hamsi de diyen fenerba...,ESFJ,sentinels,E,S,F,J


Oluşturulan dataframe shuffle edilir.

In [266]:
equal_entries_df = equal_entries_df.sample(frac=1).reset_index(drop=True)
equal_entries_df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,ramachandran,sayın bakan efsane komik sahne süper türk dizi...,INFJ,diplomats,I,N,F,J
1,tatli okur,osmanlı insan insanlık katkı hukuk,INTJ,analysts,I,N,T,J
2,unutkanfil,kız tasvir et ederken kullan kullanıl kullanıl...,ENTP,analysts,E,N,T,P
3,zipirinsan,osmanlı kadın padişah ol olmama ayıp bir şey t...,INTP,analysts,I,N,T,P
4,foxie,sevgili bulunç ara arayıp sor sormayan arkadaş...,ISFJ,sentinels,I,S,F,J
...,...,...,...,...,...,...,...,...
511713,big and beautiful eyes,the boat hat rocket müzik izle izlen izlenesi ...,INTP,analysts,I,N,T,P
511714,kissofh3ll,çaylak mesaj bekle bekleyen yazar veritabanı o...,ENTP,analysts,E,N,T,P
511715,dalga dalga inen bukleler,aile tatil sefa pezevenk anne baba sahip olduk...,ENTP,analysts,E,N,T,P
511716,fyodor fyodorovic,ravi shankar ceza mürekkep dol doldur art parç...,ISFJ,sentinels,I,S,F,J


In [0]:
X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['J/P'], random_state = 42) 

X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_count)
X_train_tf = tf_transformer.transform(X_train_count)
X_test_tf = tf_transformer.transform(X_test_count)

clf = MultinomialNB().fit(X_train_tf, y_train)
test_typeClass = y_test.values

predictions = clf.predict(X_test_tf)

In [268]:
predicted['J'] = {'actual': {'J': 0, 'P': 0}}
predicted['P'] = {'actual': {'J': 0, 'P': 0}}

predicted

{'E': {'actual': {'E': 36972, 'I': 26543}},
 'F': {'actual': {'F': 32957, 'T': 21370}},
 'I': {'actual': {'E': 27818, 'I': 38089}},
 'J': {'actual': {'J': 0, 'P': 0}},
 'N': {'actual': {'N': 20380, 'S': 13994}},
 'P': {'actual': {'J': 0, 'P': 0}},
 'S': {'actual': {'N': 16470, 'S': 23006}},
 'T': {'actual': {'F': 29540, 'T': 41037}}}

In [0]:
for i in range(len(predictions)):
  predicted[predictions[i]]['actual'][test_typeClass[i]] += 1

In [270]:
predicted

{'E': {'actual': {'E': 36972, 'I': 26543}},
 'F': {'actual': {'F': 32957, 'T': 21370}},
 'I': {'actual': {'E': 27818, 'I': 38089}},
 'J': {'actual': {'J': 33836, 'P': 24610}},
 'N': {'actual': {'N': 20380, 'S': 13994}},
 'P': {'actual': {'J': 30122, 'P': 39362}},
 'S': {'actual': {'N': 16470, 'S': 23006}},
 'T': {'actual': {'F': 29540, 'T': 41037}}}

In [0]:
accuracy = (predicted['P']['actual']['P'] + predicted['J']['actual']['J']) / len(predictions)

In [272]:
accuracy

0.5721722817165638

In [273]:
prediction_results['predicted'].update(predicted) 

prediction_results

{'predicted': {'E': {'actual': {'E': 36972, 'I': 26543}},
  'F': {'actual': {'F': 32957, 'T': 21370}},
  'I': {'actual': {'E': 27818, 'I': 38089}},
  'J': {'actual': {'J': 33836, 'P': 24610}},
  'N': {'actual': {'N': 20380, 'S': 13994}},
  'P': {'actual': {'J': 30122, 'P': 39362}},
  'S': {'actual': {'N': 16470, 'S': 23006}},
  'T': {'actual': {'F': 29540, 'T': 41037}},
  'analysts': {'actual': {'analysts': 4155,
    'diplomats': 2664,
    'explorers': 2731,
    'sentinels': 2585}},
  'diplomats': {'actual': {'analysts': 1486,
    'diplomats': 2712,
    'explorers': 1463,
    'sentinels': 1329}},
  'explorers': {'actual': {'analysts': 2106,
    'diplomats': 2160,
    'explorers': 3498,
    'sentinels': 2051}},
  'sentinels': {'actual': {'analysts': 3248,
    'diplomats': 3349,
    'explorers': 3333,
    'sentinels': 5060}}}}